In [1]:
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose
from keras.optimizers import Adam
from scipy import misc
import keras
from keras.datasets import cifar10
import os
import keras.backend.tensorflow_backend as KTF

from scipy import misc
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    KTF.set_session(tf.Session(config=config))


def predict_images(file_name, generator, noise_size, n = 10, size = 32):

    image = generator.predict(np.random.normal(size=(n*n, ) + noise_size))

    image = np.reshape(image, (n, n, size, size, 3))
    image = np.transpose(image, (0, 2, 1, 3, 4))
    image = np.reshape(image, (n*size, n*size, 3))

    image = 255 * (image + 1) / 2
    image = image.astype("uint8")
    misc.imsave(file_name, image)
def build_generator(input_shape):
    model = Sequential()

    model.add(Conv2DTranspose(512,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(3,(3,3),padding="same",activation="tanh"))
    model.summary()
    return model


def build_discriminator(input_shape):
    model = Sequential()

    model.add(Conv2D(64,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(512,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(1,(3,3),padding="same"))
    model.add(GlobalAveragePooling2D())
    model.summary()

    return model

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    LAMBA = 0.0002

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    d_loss = K.mean(K.maximum(pred_real - pred_fake + LAMBA * K.sum(K.abs(real_image-fake_image),axis=[1,2,3]), 0.0))
    g_loss = K.mean(pred_fake)

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss], d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_training_updates)

    return d_train,g_train
def train():
    set_gpu_config("0", 0.5)

    epochs = 50
    image_size = (32, 32, 3)
    noise_size = (2, 2, 32)
    batch_size = 64
    sample_size = 10
    size = 32
    sample_interval=200
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    num_of_data = x_train.shape[0]
    x_train = x_train.astype("float32")
    x_test = x_test.astype("float32")
    x_train = (x_train / 255) * 2 - 1
    x_test = (x_test / 255) * 2 - 1
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    generator = build_generator(noise_size)
    discriminator = build_discriminator(image_size)
    d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

    nb_batches = int(x_train.shape[0] / batch_size)
    global_step = 0
    for epoch in range(epochs):
        for index in range(nb_batches):
            global_step += 1
            real_images = x_train[index * batch_size:(index + 1) * batch_size]
            d_loss, = d_train([real_images, 1])
            g_loss, = g_train([real_images, 1])
            print("[{0}/{1}] [{2}_{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(epoch, epochs, epoch, global_step, d_loss,
                                                                              g_loss))
            if global_step % sample_interval == 0:
                image = generator.predict(np.random.normal(size=(sample_size * sample_size,) + noise_size))
                image = np.reshape(image, (sample_size, sample_size, size, size, 3))
                image = np.transpose(image, (0, 2, 1, 3, 4))
                image = np.reshape(image, (sample_size * size, sample_size * size, 3))

                image = 255 * (image + 1) / 2
                image = image.astype("uint8")
                if not os.path.isdir('images_lsgan_cifar10'):
                    os.mkdir('images_lsgan_cifar10')
                misc.imsave("images_lsgan_cifar10/e{0}_step{1}img.png".format(epoch,global_step), image)

if __name__ == '__main__':
    train()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_1 (Conv2DTr (None, 4, 4, 512)         147968    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 8, 8, 256)         1179904   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 16, 16, 128)       295040    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 64)        73792     
__________

[0/50] [0_111] d_loss: 0.0, g_loss: -0.2662
[0/50] [0_112] d_loss: 0.0, g_loss: -0.5071
[0/50] [0_113] d_loss: 0.004253, g_loss: 0.05196
[0/50] [0_114] d_loss: 0.0001298, g_loss: -0.1437
[0/50] [0_115] d_loss: 7.831e-05, g_loss: -0.451
[0/50] [0_116] d_loss: 0.05781, g_loss: 0.2471
[0/50] [0_117] d_loss: 0.001466, g_loss: 0.1574
[0/50] [0_118] d_loss: 0.0, g_loss: -0.2375
[0/50] [0_119] d_loss: 0.02484, g_loss: 0.09467
[0/50] [0_120] d_loss: 0.0002598, g_loss: -0.07514
[0/50] [0_121] d_loss: 0.0005599, g_loss: -0.1678
[0/50] [0_122] d_loss: 0.0002452, g_loss: -0.3069
[0/50] [0_123] d_loss: 0.009333, g_loss: 0.1686
[0/50] [0_124] d_loss: 3.116e-05, g_loss: -0.01085
[0/50] [0_125] d_loss: 0.001142, g_loss: 0.06858
[0/50] [0_126] d_loss: 1.96e-06, g_loss: -0.1329
[0/50] [0_127] d_loss: 0.01486, g_loss: 0.2673
[0/50] [0_128] d_loss: 0.0001761, g_loss: 0.06803
[0/50] [0_129] d_loss: 9.632e-05, g_loss: -0.129
[0/50] [0_130] d_loss: 0.01272, g_loss: -0.2471
[0/50] [0_131] d_loss: 0.004777, g_

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:138: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


[0/50] [0_202] d_loss: 0.03805, g_loss: -0.003795
[0/50] [0_203] d_loss: 0.009636, g_loss: -0.05887
[0/50] [0_204] d_loss: 0.01477, g_loss: 0.1001
[0/50] [0_205] d_loss: 0.02459, g_loss: -0.7847
[0/50] [0_206] d_loss: 0.007976, g_loss: -1.166
[0/50] [0_207] d_loss: 0.04214, g_loss: 0.7839
[0/50] [0_208] d_loss: 0.006978, g_loss: 0.4387
[0/50] [0_209] d_loss: 0.009799, g_loss: -0.09439
[0/50] [0_210] d_loss: 0.005353, g_loss: 0.2389
[0/50] [0_211] d_loss: 0.0138, g_loss: 0.3198
[0/50] [0_212] d_loss: 0.01958, g_loss: 0.09624
[0/50] [0_213] d_loss: 0.0154, g_loss: -0.6957
[0/50] [0_214] d_loss: 0.01176, g_loss: -0.2104
[0/50] [0_215] d_loss: 0.01017, g_loss: -0.2824
[0/50] [0_216] d_loss: 0.02312, g_loss: 0.5763
[0/50] [0_217] d_loss: 0.03991, g_loss: -1.104
[0/50] [0_218] d_loss: 0.02299, g_loss: -0.562
[0/50] [0_219] d_loss: 0.008073, g_loss: -0.02457
[0/50] [0_220] d_loss: 0.005547, g_loss: 0.1558
[0/50] [0_221] d_loss: 0.0001523, g_loss: -0.004855
[0/50] [0_222] d_loss: 0.0, g_loss: 

...

[49/50] [49_38289] d_loss: 0.07367, g_loss: 1.394
[49/50] [49_38290] d_loss: 0.0868, g_loss: 1.485
[49/50] [49_38291] d_loss: 0.07501, g_loss: 1.733
[49/50] [49_38292] d_loss: 0.05951, g_loss: 1.796
[49/50] [49_38293] d_loss: 0.1291, g_loss: 1.707
[49/50] [49_38294] d_loss: 0.175, g_loss: 1.568
[49/50] [49_38295] d_loss: 0.09404, g_loss: 1.485
[49/50] [49_38296] d_loss: 0.07092, g_loss: 1.609
[49/50] [49_38297] d_loss: 0.09932, g_loss: 1.339
[49/50] [49_38298] d_loss: 0.04385, g_loss: 1.206
[49/50] [49_38299] d_loss: 0.08641, g_loss: 1.255
[49/50] [49_38300] d_loss: 0.1054, g_loss: 1.238
[49/50] [49_38301] d_loss: 0.1114, g_loss: 1.485
[49/50] [49_38302] d_loss: 0.09823, g_loss: 1.248
[49/50] [49_38303] d_loss: 0.1261, g_loss: 1.369
[49/50] [49_38304] d_loss: 0.1707, g_loss: 1.511
[49/50] [49_38305] d_loss: 0.09537, g_loss: 1.497
[49/50] [49_38306] d_loss: 0.04041, g_loss: 1.421
[49/50] [49_38307] d_loss: 0.07701, g_loss: 1.488
[49/50] [49_38308] d_loss: 0.06212, g_loss: 1.532
[49/50] 

[49/50] [49_38463] d_loss: 0.08244, g_loss: 1.306
[49/50] [49_38464] d_loss: 0.09983, g_loss: 1.363
[49/50] [49_38465] d_loss: 0.08618, g_loss: 1.3
[49/50] [49_38466] d_loss: 0.1254, g_loss: 1.52
[49/50] [49_38467] d_loss: 0.0566, g_loss: 1.429
[49/50] [49_38468] d_loss: 0.1068, g_loss: 1.296
[49/50] [49_38469] d_loss: 0.1435, g_loss: 1.379
[49/50] [49_38470] d_loss: 0.08599, g_loss: 1.292
[49/50] [49_38471] d_loss: 0.07542, g_loss: 1.364
[49/50] [49_38472] d_loss: 0.07477, g_loss: 1.394
[49/50] [49_38473] d_loss: 0.08458, g_loss: 1.501
[49/50] [49_38474] d_loss: 0.1055, g_loss: 1.367
[49/50] [49_38475] d_loss: 0.1114, g_loss: 1.186
[49/50] [49_38476] d_loss: 0.07541, g_loss: 1.175
[49/50] [49_38477] d_loss: 0.1068, g_loss: 1.297
[49/50] [49_38478] d_loss: 0.07865, g_loss: 1.381
[49/50] [49_38479] d_loss: 0.1039, g_loss: 1.612
[49/50] [49_38480] d_loss: 0.06209, g_loss: 1.542
[49/50] [49_38481] d_loss: 0.05519, g_loss: 1.527
[49/50] [49_38482] d_loss: 0.07675, g_loss: 1.544
[49/50] [49

[49/50] [49_38636] d_loss: 0.1255, g_loss: 1.511
[49/50] [49_38637] d_loss: 0.09417, g_loss: 1.709
[49/50] [49_38638] d_loss: 0.1234, g_loss: 1.437
[49/50] [49_38639] d_loss: 0.08196, g_loss: 1.179
[49/50] [49_38640] d_loss: 0.06433, g_loss: 1.258
[49/50] [49_38641] d_loss: 0.08285, g_loss: 1.174
[49/50] [49_38642] d_loss: 0.1089, g_loss: 1.396
[49/50] [49_38643] d_loss: 0.09591, g_loss: 1.433
[49/50] [49_38644] d_loss: 0.09471, g_loss: 1.471
[49/50] [49_38645] d_loss: 0.07391, g_loss: 1.43
[49/50] [49_38646] d_loss: 0.05415, g_loss: 1.525
[49/50] [49_38647] d_loss: 0.1055, g_loss: 1.525
[49/50] [49_38648] d_loss: 0.1035, g_loss: 1.696
[49/50] [49_38649] d_loss: 0.1078, g_loss: 1.506
[49/50] [49_38650] d_loss: 0.08996, g_loss: 1.299
[49/50] [49_38651] d_loss: 0.07272, g_loss: 1.41
[49/50] [49_38652] d_loss: 0.1122, g_loss: 1.498
[49/50] [49_38653] d_loss: 0.07102, g_loss: 1.471
[49/50] [49_38654] d_loss: 0.08201, g_loss: 1.418
[49/50] [49_38655] d_loss: 0.07519, g_loss: 1.572
[49/50] [

[49/50] [49_38812] d_loss: 0.06903, g_loss: 1.424
[49/50] [49_38813] d_loss: 0.1724, g_loss: 1.541
[49/50] [49_38814] d_loss: 0.07664, g_loss: 1.595
[49/50] [49_38815] d_loss: 0.1027, g_loss: 1.551
[49/50] [49_38816] d_loss: 0.08957, g_loss: 1.58
[49/50] [49_38817] d_loss: 0.1043, g_loss: 1.579
[49/50] [49_38818] d_loss: 0.1127, g_loss: 1.671
[49/50] [49_38819] d_loss: 0.07974, g_loss: 1.649
[49/50] [49_38820] d_loss: 0.08303, g_loss: 1.643
[49/50] [49_38821] d_loss: 0.08998, g_loss: 1.702
[49/50] [49_38822] d_loss: 0.08867, g_loss: 1.606
[49/50] [49_38823] d_loss: 0.09446, g_loss: 1.662
[49/50] [49_38824] d_loss: 0.1181, g_loss: 1.539
[49/50] [49_38825] d_loss: 0.199, g_loss: 1.608
[49/50] [49_38826] d_loss: 0.09829, g_loss: 1.524
[49/50] [49_38827] d_loss: 0.1266, g_loss: 1.595
[49/50] [49_38828] d_loss: 0.1014, g_loss: 1.675
[49/50] [49_38829] d_loss: 0.08142, g_loss: 1.681
[49/50] [49_38830] d_loss: 0.08493, g_loss: 1.614
[49/50] [49_38831] d_loss: 0.04873, g_loss: 1.628
[49/50] [4

[49/50] [49_38988] d_loss: 0.04874, g_loss: 1.186
[49/50] [49_38989] d_loss: 0.1058, g_loss: 1.289
[49/50] [49_38990] d_loss: 0.09905, g_loss: 1.242
[49/50] [49_38991] d_loss: 0.06993, g_loss: 1.328
[49/50] [49_38992] d_loss: 0.1064, g_loss: 1.543
[49/50] [49_38993] d_loss: 0.1192, g_loss: 1.464
[49/50] [49_38994] d_loss: 0.05628, g_loss: 1.448
[49/50] [49_38995] d_loss: 0.06507, g_loss: 1.369
[49/50] [49_38996] d_loss: 0.09657, g_loss: 1.386
[49/50] [49_38997] d_loss: 0.1268, g_loss: 1.378
[49/50] [49_38998] d_loss: 0.1174, g_loss: 1.338
[49/50] [49_38999] d_loss: 0.06952, g_loss: 1.302
[49/50] [49_39000] d_loss: 0.08655, g_loss: 1.5
[49/50] [49_39001] d_loss: 0.1252, g_loss: 1.555
[49/50] [49_39002] d_loss: 0.06603, g_loss: 1.434
[49/50] [49_39003] d_loss: 0.1359, g_loss: 1.841
[49/50] [49_39004] d_loss: 0.05948, g_loss: 1.602
[49/50] [49_39005] d_loss: 0.08971, g_loss: 1.517
[49/50] [49_39006] d_loss: 0.1225, g_loss: 1.598
[49/50] [49_39007] d_loss: 0.09986, g_loss: 1.522
[49/50] [4